In [1]:
!pip3 install crunch-cli --upgrade

  Using cached click-8.1.3-py3-none-any.whl (96 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 739.8 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 4.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.9/38.9 MB 5.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 7.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.4 MB/s eta 0:00:00
  Using cached pytz-2023.3-py2.py3-none-any.whl (502 kB)
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.0 MB/s eta 0:00:00
  Created wheel for gitignorefile: filename=gitignorefile-1.1.2-py3-none-any.whl size=6672 sha256=eb14ab7ed1982a3bae89badf76337b3aee9b9d914e7f1a9a8d31c3288bccc139
  Stored in di

In [2]:
!crunch --notebook setup shrill-mohamed --token RKWcWWZlh0F23CITn46MN7sZUyzNQa5m4mBwP6cFcR1wc5HJSYW7wBS5AnvTI23e
%cd shrill-mohamed

loaded inline runner with module: <module '__main__'>
extract shrill-mohamed/main.py
extract shrill-mohamed/notebook.ipynb
extract shrill-mohamed/requirements.txt
download data/X_train.parquet from https://datacrunch-com.s3.eu-west-1.amazonaws.com/production/adialab/data-releases/1/X_train.parquet
download data/y_train.parquet from https://datacrunch-com.s3.eu-west-1.amazonaws.com/production/adialab/data-releases/1/y_train.parquet
download data/X_test.parquet from https://datacrunch-com.s3.eu-west-1.amazonaws.com/production/adialab/data-releases/1/X_test_reduced.parquet
                                                                                
---
Success! Your environment has been correctly setup.
Next recommended actions:
 - To get inside your workspace directory, run: cd shrill-mohamed
 - To see all of the available commands of the CrunchDAO CLI, run: crunch --help
/home/ptl/internship/all_machine_learning/ranking/LambdaMart/shrill-mohamed


In [1]:
import crunch
crunch = crunch.load_notebook(__name__)

loaded inline runner with module: <module '__main__'>


In [2]:
%cd shrill-mohamed
X_train, y_train, X_test = crunch.load_data()

/home/ptl/internship/all_machine_learning/ranking/LambdaMart/shrill-mohamed
download data/X_train.parquet from https://datacrunch-com.s3.eu-west-1.amazonaws.com/production/adialab/data-releases/1/X_train.parquet
already exists: file length match
download data/y_train.parquet from https://datacrunch-com.s3.eu-west-1.amazonaws.com/production/adialab/data-releases/1/y_train.parquet
already exists: file length match
download data/X_test.parquet from https://datacrunch-com.s3.eu-west-1.amazonaws.com/production/adialab/data-releases/1/X_test_reduced.parquet
already exists: file length match


In [3]:
X_train_orig = X_train.copy()
y_train_orig = y_train.copy()

In [4]:
slicing = int(X_train.date.max()*0.8)
slicing_index = X_train[X_train["date"]==slicing].index.min()


In [5]:
X_train = X_train_orig.iloc[:slicing_index,:]
y_train = y_train_orig.iloc[:slicing_index,:]
X_eval = X_train_orig.iloc[slicing_index:,:]
y_eval = y_train_orig.iloc[slicing_index:,:]
print(len(X_train),len(y_train))
print(len(X_train_orig),len(y_train_orig))
print(len(X_eval),len(y_eval))

535729 535729
742670 742670
206941 206941


In [8]:
import numpy as np
import pandas as pd

def get_data(x_train,y_train):
	data = []
	for index,line in enumerate(x_train.T):
		new_arr = []
		score = y_train.iloc[index,-1]
		q_id = x_train.iloc[index,0]
		new_arr.append(int(score))
		new_arr.append(int(q_id))
		arr = x_train.iloc[index,2:]
		for el in arr:
			new_arr.append(el)
		data.append(new_arr)
	return np.array(data)

In [9]:
from lambdamart import LambdaMART


In [10]:
def main():
	total_ndcg = 0.0
	for i in [1]:
		print('start Fold ' + str(i))
		training_data = get_data(X_train.iloc[:1000,:],y_train.iloc[:1000,:])
		test_data = get_data(X_eval.iloc[:1000,:],y_eval.iloc[:1000,:])
		model = LambdaMART(training_data, 100, 0.01, 'sklearn')
		model.fit()
		#model.save('lambdamart_model_%d' % (i))
		# model = LambdaMART()
		# model.load('lambdamart_model.lmart')
		#average_ndcg, predicted_scores = model.validate(test_data, 10)
		#print(average_ndcg)
		#total_ndcg += average_ndcg
	#total_ndcg /= 5.0
	#print('Original average ndcg at 10 is: ' + str(total_ndcg))
	#total_ndcg = 0.0
	return model
	'''for i in [1,2,3,4,5]:
		print('start Fold ' + str(i))
		training_data = get_data(X_train,y_train)
		test_data = get_data(X_eval,y_eval)
		model = LambdaMART(training_data, 300, 0.001, 'original')
		model.fit()
		model.save('lambdamart_model_sklearn_%d' % (i))
		# model = LambdaMART()
		# model.load('lambdamart_model.lmart')
		average_ndcg, predicted_scores = model.validate(test_data, 10)
		print(average_ndcg)
		total_ndcg += average_ndcg
	total_ndcg /= 5.0
	print( 'Sklearn average ndcg at 10 is: ' + str(total_ndcg))'''
model = main()

start Fold 1
Tree 0
Tree 1
Tree 2
Tree 3
Tree 4
Tree 5
Tree 6
Tree 7
Tree 8
Tree 9
Tree 10
Tree 11
Tree 12
Tree 13
Tree 14
Tree 15
Tree 16
Tree 17
Tree 18
Tree 19
Tree 20
Tree 21
Tree 22
Tree 23
Tree 24
Tree 25
Tree 26
Tree 27
Tree 28
Tree 29
Tree 30
Tree 31
Tree 32
Tree 33
Tree 34
Tree 35
Tree 36
Tree 37
Tree 38
Tree 39
Tree 40
Tree 41
Tree 42
Tree 43
Tree 44
Tree 45
Tree 46
Tree 47
Tree 48
Tree 49
Tree 50
Tree 51
Tree 52
Tree 53
Tree 54
Tree 55
Tree 56
Tree 57
Tree 58
Tree 59
Tree 60
Tree 61
Tree 62
Tree 63
Tree 64
Tree 65
Tree 66
Tree 67
Tree 68
Tree 69
Tree 70
Tree 71
Tree 72
Tree 73
Tree 74
Tree 75
Tree 76
Tree 77
Tree 78
Tree 79
Tree 80
Tree 81
Tree 82
Tree 83
Tree 84
Tree 85
Tree 86
Tree 87
Tree 88
Tree 89
Tree 90
Tree 91
Tree 92
Tree 93
Tree 94
Tree 95
Tree 96
Tree 97
Tree 98
Tree 99


In [14]:
def get_test_data(x_train):
	data = []
	for index,line in enumerate(x_train.T):
		new_arr = []
		q_id = x_train.iloc[index,0]
		new_arr.append(int(q_id))
		arr = x_train.iloc[index,2:]
		for el in arr:
			new_arr.append(el)
		data.append(new_arr)
	return np.array(data)
test_data = get_test_data(X_eval.iloc[:10,:])
model.predict(test_data)


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
from scipy.stats import spearmanr
score = spearmanr(y_test_local["y"], y_test_local_pred["value"])[0] * 100
print(f"Spearman's correlation {score}")